# TWO OPTIONS

### #0.1 Load mask rcnn results

In [ ]:
import json
import os
import shutil
import glob

In [ ]:
results = json.load(open('/root/data/headtail/detection_coco_results2.json'))

In [ ]:
print("Number of images: {}".format(len(results["images"])))
print("Number of annotations: {}".format(len(results["annotations"])))
print("Number of unique annotations: {}".format(len(list(set([k['image_id'] for k in results["annotations"]])))))

In [ ]:
# create image dic for faster search
image_dic = {}
for img in results["images"]:
    image_dic[img["id"]] = img

In [ ]:
# create ann dic for faster search
ann_dic = {}
for ann in results["annotations"]:
    ann_dic[ann["image_id"]] = ann

In [ ]:
# list all the images paths with detections
paths = []
for ann in results['annotations']:
    img_path = image_dic[ann['image_id']]["local_path"]
    paths.append((img_path, ann["image_id"]))
print(len(paths))

In [ ]:
# good pairs manually filtered
good_ts = []
for image_path in glob.glob('/root/data/headtail/good_pairs/*'):
    ts = int(os.path.basename(image_path).split('.')[0].split('_')[-1])
    good_ts.append(ts)
good_ts = list(set(good_ts))

In [ ]:
# timestamps dict
timestamps = {}
for path in paths:
    path0 = path[0]
    ts = int(os.path.basename(path0).split('.')[0].split('_')[-1])
    if ts not in good_ts:
        continue
    side = os.path.basename(path0).split('.')[0].split('_')[0]
    if ts not in timestamps:
        timestamps[ts] = {}
    timestamps[ts][side] = path0
    timestamps[ts][side + "_id"] = path[1]

In [ ]:
print("number of pairs: {}".format(len([v for (k, v) in timestamps.items() if "right" in v and "left" in v])))

In [ ]:
good_pairs = [v for (k, v) in timestamps.items() if "right" in v and "left" in v]

In [ ]:
# for gp in good_pairs:
#     shutil.copy(gp['left'], os.path.join('/root/data/headtail/good_pairs/', os.path.basename(gp['left'])))
#     shutil.copy(gp['right'], os.path.join('/root/data/headtail/good_pairs/', os.path.basename(gp['right'])))

so for 406 pairs we have full fish on both side

### #1 Display some results

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw
from matplotlib.patches import Rectangle, Polygon
import numpy as np
from pycocotools.mask import decode

In [ ]:
# gp = {'left': '/root/data/rnd/small_pen_data_collection/sotra-small-pen/pen-1/2018-10-01/181001010007_rectified/left_sotra-small-pen_0_1538488432310.jpg',
#   'left_id': 2490,
#   'right': '/root/data/rnd/small_pen_data_collection/sotra-small-pen/pen-1/2018-10-01/181001010007_rectified/right_sotra-small-pen_0_1538488432310.jpg',
#   'right_id': 2763}

In [ ]:
random_timestamp = np.random.choice(good_pairs)
# random_timestamp = gp
# left side
left_img = Image.open(random_timestamp["right"])
left_ann = ann_dic[random_timestamp["right_id"]]
left_bbox = left_ann['bbox']
left_rec = Rectangle((left_bbox[1], left_bbox[0]), left_bbox[3]-left_bbox[1], left_bbox[2]-left_bbox[0], linewidth=2,edgecolor='w',linestyle="--",facecolor='none')
seg = left_ann['segmentation'][0]
left_poly = np.array(seg).reshape((int(len(seg)/2), 2))
left_mask = Polygon(left_poly)

# right_side
right_img = Image.open(random_timestamp["left"])
right_ann = ann_dic[random_timestamp["left_id"]]
right_bbox = right_ann['bbox']
right_rec = Rectangle((right_bbox[1], right_bbox[0]), right_bbox[3]-right_bbox[1], right_bbox[2]-right_bbox[0], linewidth=2,edgecolor='w',linestyle="--",facecolor='none')
seg = right_ann['segmentation'][0]
poly = np.array(seg).reshape((int(len(seg)/2), 2))
right_mask = Polygon(poly)


f, ax = plt.subplots(1, 2, figsize=(20, 10))
ax[0].imshow(left_img)
ax[0].add_patch(left_rec)
ax[0].add_patch(left_mask)

ax[1].imshow(right_img)
ax[1].add_patch(right_rec)
ax[1].add_patch(right_mask)

plt.show()

### #2 Calculate centroids, depth, length

In [ ]:
from sklearn.cluster import DBSCAN, KMeans

In [ ]:
focal_length = 0.0107
baseline = 0.135
pixel_size_m = 3.45 * 1e-6 
focal_length_pixel = focal_length / pixel_size_m
image_sensor_width = 0.01412
image_sensor_height = 0.01412

In [ ]:
def convert_to_world_point(x, y, d):
    image_center_x = 3000 / 2.0 #depth_map.shape[1] / 2.0
    image_center_y = 4096 / 2.0# depth_map.shape[0] / 2.0
    px_x = x - image_center_x
    px_z = image_center_y - y

    sensor_x = px_x * (image_sensor_width / 3000)
    sensor_z = px_z * (image_sensor_height / 4096)
    
    # d = depth_map[y, x]
    world_y = d
    world_x = (world_y * sensor_x) / focal_length
    world_z = (world_y * sensor_z) / focal_length
    return (world_x, world_y, world_z)

In [ ]:
random_state = 170

In [ ]:
ground_truth = {'181001010001': 0.695,
 '1810010100010': 0.655,
 '181001010002': 0.75,
 '181001010003': 0.585,
 '181001010004': 0.625,
 '181001010005': 0.685,
 '181001010006': 0.645,
 '181001010007': 0.535,
 '181001010008': 0.66,
 '181001010009': 0.56,
 '181001010010': 0.655}

In [ ]:
results = {}
for (i, random_timestamp) in enumerate(good_pairs):
    if i % 10 == 0:
        print('{}/{}'.format(i, len(good_pairs)))
    
    experience = random_timestamp["right"].split('/')[-2].split('_')[0]
    if experience not in results:
        results[experience] = []
    # left side
    left_img = Image.open(random_timestamp["right"])
    left_ann = ann_dic[random_timestamp["right_id"]]
    left_bbox = left_ann['bbox']
    left_rec = Rectangle((left_bbox[1], left_bbox[0]), left_bbox[3]-left_bbox[1], left_bbox[2]-left_bbox[0], linewidth=2,edgecolor='w',linestyle="--",facecolor='none')

    seg = left_ann['segmentation'][0]
    left_poly = np.array(seg).reshape((int(len(seg)/2), 2))
#     left_X = left_poly
    lp = [(r[0], r[1]) for r in left_poly]
    left_mask = Image.new('L', (4096, 3000), 0)
    ImageDraw.Draw(left_mask).polygon(lp, outline=1, fill=1)
    left_mask = np.array(left_mask)
    left_X = np.stack([np.nonzero(left_mask)[0], np.nonzero(left_mask)[1]], axis=1)
    left_X = left_X[::100, :]
    
    left_y_pred = KMeans(n_clusters=6, random_state=random_state).fit_predict(left_X)
    centroids = []
    for label in np.unique(left_y_pred):
        x_mean = np.mean(left_X[left_y_pred==label, 0])
        y_mean = np.mean(left_X[left_y_pred==label, 1])
        centroids.append((x_mean, y_mean))
    left_centroids = np.array(centroids)
    left_centroids = left_centroids[left_centroids[:,1].argsort()]

    # right side
    right_img = Image.open(random_timestamp["left"])
    right_ann = ann_dic[random_timestamp["left_id"]]
    right_bbox = right_ann['bbox']
    right_rec = Rectangle((right_bbox[1], right_bbox[0]), right_bbox[3]-right_bbox[1], right_bbox[2]-right_bbox[0], linewidth=2,edgecolor='w',linestyle="--",facecolor='none')

    seg = right_ann['segmentation'][0]
    right_poly = np.array(seg).reshape((int(len(seg)/2), 2))
#     right_X = right_poly
    rp = [(r[0], r[1]) for r in right_poly]
    right_mask = Image.new('L', (4096, 3000), 0)
    ImageDraw.Draw(right_mask).polygon(rp, outline=1, fill=1)
    right_mask = np.array(right_mask)
    right_X = np.stack([np.nonzero(right_mask)[0], np.nonzero(right_mask)[1]], axis=1)
    right_X = right_X[::100, :]
    
    right_y_pred = KMeans(n_clusters=6, random_state=random_state).fit_predict(right_X)
    centroids = []
    for label in np.unique(right_y_pred):
        x_mean = np.mean(right_X[right_y_pred==label, 0])
        y_mean = np.mean(right_X[right_y_pred==label, 1])
        centroids.append((x_mean, y_mean))
    right_centroids = np.array(centroids)
    right_centroids = right_centroids[right_centroids[:,1].argsort()]
    
    if i % 50 == 0:
        f, ax = plt.subplots(1,2, figsize=(20, 10))
        ax[0].imshow(left_img)
        # ax[0].scatter(left_X[:, 0], left_X[:, 1], c=left_y_pred)
        ax[0].scatter(left_centroids[[0, -1], 1], left_centroids[[0, -1], 0], c='r')
        # ax[0].scatter(left_centroids[:, 1], left_centroids[:, 0], c='r')
        
        ax[1].imshow(right_img)
        # ax[1].scatter(right_X[:, 0], right_X[:, 1], c=right_y_pred)
        ax[1].scatter(right_centroids[[0, -1], 1], right_centroids[[0, -1], 0], c='r')
        plt.show()

    
    # estimate disparities
    # print(left_centroids)
    # print(right_centroids)
    disparities = left_centroids[[0, -1], 1] - right_centroids[[0, -1], 1]
    # print(disparities)

    # estimate depth
    depth = focal_length_pixel*baseline / np.array(disparities)
    # print(depth)

    # calculate world coordinate of point 0 
    world0 = convert_to_world_point(left_centroids[0][0], left_centroids[0][1], depth[0])
    # print(world0)

    # calculate world coordinate of point 1 
    world1 = convert_to_world_point(left_centroids[-1][0], left_centroids[-1][1], depth[1])
    # print(world1)

    # print length
    predicted_length = np.linalg.norm(np.array(world0) - np.array(world1))
    results[experience].append(predicted_length)
    
#     print("example: {}".format(i))
#     print("predicted length: {}".format(predicted_length))
#     print("ground truth length: {}".format(ground_truth[experience]))
#     print("#"*100)

In [ ]:
for (i, exp) in enumerate(results.keys()):
    plt.scatter(np.zeros_like(np.array(results[exp]))+i, np.array(results[exp]))
    plt.plot([i-0.3, i+0.3], [ground_truth[exp]]*2, color='k')
plt.xticks(range(10), list(results.keys()), rotation=70)
plt.ylim([0, 2])
plt.show()

In [ ]:
density = 1e6 # g per cubic meter

In [ ]:
ground_truth_weights = {'181001010001':4315,
'181001010002':5981,
'181001010003':2773,
'181001010004':3164,
'181001010005':4480,
'181001010006':2559,
'181001010007':2121,
'181001010008':3949,
'181001010009':1957,
'181001010010':2997}

In [ ]:
for (i, exp) in enumerate(results.keys()):
    plt.scatter(np.zeros_like(np.array(results[exp]))+i, np.array(results[exp])**3*density/44.0)
    plt.plot([i-0.3, i+0.3], [ground_truth_weights[exp]]*2, color='k')
plt.xticks(range(10), list(results.keys()), rotation=70)
plt.ylim([0,10000])
plt.show()

In [ ]:
ratios = []
for (i, exp) in enumerate(results.keys()):
    median_weight = np.median(np.array(results[exp])**3*density)
    true_weights = ground_truth_weights[exp]
    ratio = median_weight/true_weights
    print(ratio)
    ratios.append(ratio)

# load ground truth results

In [ ]:
import json
import os

In [ ]:
ground_truth = json.load(open('/root/thomas/github/cv_research/thomas/full_pipeline/small_pen/head_tails_segmentation.json'))
print("total number of images: {}".format(len(ground_truth)))

In [ ]:
full = []
for gt in ground_truth:
    label = gt['Label']
    if "Head" in label and "Tail" in label:
        if "sotra" in gt['External ID']:
            full.append(gt)
print("Total number of full images from small pen: {}".format(len(full)))

In [ ]:
# check that the left is not too much on the left so the right one is also full
full_pairs = []
for ff in full:
    label = ff['Label']
    total_poly = label['Head'][0]['geometry'] + label['Tail'][0]['geometry']
    array = np.array([[k['x'], k['y']] for k in total_poly])
    y1, y2 = array[:, 0].min(), array[:, 0].max()
    if y1 > 1000:
        full_pairs.append(ff)
print("Total number of full pairs: {}".format(len(full_pairs)))